In [48]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns


In [49]:
###########################################################################
file_path = "/Users/sungjune/Downloads/PJ3/main_dataset(2015~2021).csv"
EDA_FILE_NAME = "도매_EDA_완료_new_v3.0.csv"
POP_LOCA_FILE_NAME = "지역_추출_도매_데이터_new_v3.0.csv"
###########################################################################
pprc_dict = {}
raw_data = pd.read_csv(file_path)
raw_data.drop(columns=["Unnamed: 0", "ROW_NUM"], inplace=True)
feature_list = \
    ["DELNG_DE", "PBLMNG_WHSAL_MRKT_NM", "CPR_NM", 
     "SPCIES_NM", "CPR_USE_SPCIES_NM", "PRICE", "DELNGBUNDLE_QY", 
     "STNDRD", "DELNG_QY", "SANJI_NM", "CPR_USE_SANJI_NM"
    ]
select_data = raw_data[feature_list]

In [50]:
# a = select_data.loc[
#     (select_data["PRICE"] >= 10000)
#     & (select_data.DELNGBUNDLE_QY == 1)
#     & (select_data.SPCIES_NM == "대파(일반)")
#     ]

# b = select_data.loc[
#     (select_data.DELNGBUNDLE_QY == 1)
#     & (select_data.SPCIES_NM == "대파(일반)")
# ]
# print(len(a))
# print(format(len(a) / len(b) * 100, ".2f")) 
# ((a["PBLMNG_WHSAL_MRKT_NM"].value_counts() / b["PBLMNG_WHSAL_MRKT_NM"].value_counts()).apply(lambda x: round(x, 4)) * 100).sort_values()

# # select_data.loc[select_data.DELNGBUNDLE_QY == 1].sort_values("PRICE", ascending=False).iloc[600:650]

In [51]:
# def change_columns_name(df):
#     df.rename(columns = {'DELNG_DE' : '경락일',
#                             'PBLMNG_WHSAL_MRKT_NM' : '공영 도매시장명',
#                             'PBLMNG_WHSAL_MRKT_CD' : '공영 도매시장 코드',
#                             'CPR_NM' : '법인명',
#                             'CPR_CD' : '법인코드',
#                             'RISENO' : '원표번호',
#                             'BIDTIME' : '경매시간',
#                             'AUC_SE_NM' : '경매구분',
#                             'AUC_SE_CODE' : '경매구분코드',
#                             'ORGNO' : '일련번호',
#                             'PRDLST_NM' : '품목명',
#                             'PRDLST_CD' : '품목코드',
#                             'SPCIES_NM' : '품종명',
#                             'SPCIES_CD' : '품종코드',
#                             'CPR_USE_PRDLST_NM' : '법인사용품명',
#                             'CPR_USE_SPCIES_NM' : '법인사용품종명',
#                             'PRICE' : '가격',
#                             'DELNGBUNDLE_QY' : '거래단량',
#                             'STNDRD' : '규격',
#                             'STNDRD_CD' : '규격코드',
#                             'GRAD' : '등급',
#                             'GRAD_CD' : '등급코드',
#                             'SHIPMNT_SE_CODE' : '출하구분코드',
#                             'SHIPMNT_SE_NM' : '출하구분명',
#                             'SANJI_CD' : '산지코드',
#                             'SANJI_NM' : '산지명',
#                             'CPR_USE_SANJI_CD' : '법인산지코드',
#                             'CPR_USE_SANJI_NM' : '법인산지명',
#                             'DELNG_QY' : '거래량'
#                             }, inplace = True)

# def pick_columns(df):
#     #df = df[['경락일','공영 도매시장명','법인명','경매구분','품목명','품종코드','법인사용품종명',
#     #'가격','거래단량','규격','산지명','법인산지명','거래량']]

#     df.drop(['Unnamed: 0','ROW_NUM','공영 도매시장 코드','법인코드','원표번호','경매시간',
#     '경매구분코드','일련번호','품목코드','품종코드','규격코드','등급','등급코드',
#     '출하구분코드','출하구분명','산지코드','법인산지코드'],axis=1,inplace=True)

In [52]:
# 연도 설정
# drop_index = raw_data[raw_data["DELNG_DE"] < 20160101].index
# raw_data.drop(drop_index, inplace=True)
# raw_data.reset_index(drop=True, inplace=True)

In [53]:
def pprc_default(ds):
    """
    preprocessing
    """

    # 기본 결측치 및 이상치 수정 부분
    data1 = ds.copy()

    # 전처리 내역 저장
    global pprc_dict

    # 원본 데이터 개수
    total_n = len(data1)
    pprc_dict["Raw data 개수(B)"] = format(total_n, ',')

    # 산지가 없는 값 정보 출력
    na_n = len(data1.loc[data1["SANJI_NM"].isna()])
    total_n = len(data1)
    print(f"""산지가 없는 값 비율: {format(na_n / total_n * 100, ".2f")}%""")
    print(f"""산지가 없는 값 개수: {format(na_n, ',')}개""")
    pprc_dict["산지가 없는 값(B)"] = format(na_n, ',')

    # 산지가 결측이면 법인산지로 대치함
    data1.loc[data1["SANJI_NM"].isna(), "SANJI_NM"] = \
        data1.loc[data1["SANJI_NM"].isna(), "CPR_USE_SANJI_NM"]

    # 산지 결측치 처리 후 정보 출력
    na_n_r = len(data1.loc[data1["SANJI_NM"].isna()])
    print(f"""산지가 없는 값 {format((na_n - na_n_r), ',')}개 대체""")
    print(f"""결측치 처리 후 산지가 없는 값 개수: {format(na_n_r, ',')}개 삭제""")
    pprc_dict["산지가 없는 값 대체(A)"] = format((na_n - na_n_r), ',')
    pprc_dict["산지가 없는 값 삭제(A)"] = format(na_n_r, ',')

    # 산지를 찾을 수 없으면 삭제함
    drop_index = data1.loc[data1["SANJI_NM"].isna(), "SANJI_NM"].index
    data1.drop(drop_index, inplace=True)

    # 가격, 거래단량, 거래량이 하나라도 0인 값은 삭제함
    drop_index = data1.loc[
                    (data1["PRICE"] == 0)
                    | (data1["DELNGBUNDLE_QY"] == 0)
                    | (data1["DELNG_QY"] == 0)
                ].index
    data1.drop(drop_index, inplace=True)
    print(
        f"가격, 거래단량, 거래량이 하나라도 0인 값 {format(len(drop_index), ',')}개 삭제"
    )
    pprc_dict["가격, 거래단량, 거래량이 하나라도 0인 값 삭제(A)"] = \
        format(len(drop_index), ',')

    # 대파(일반)만 가져옴
    temp = data1.loc[data1["SPCIES_NM"] == "대파(일반)"]
    print("대파(일반)의 비율: {0:.2f}%".format(len(temp) / len(data1) * 100))
    pprc_dict["대파(일반) 아닌 값 삭제(A)"] = format((len(data1) - len(temp)), ',')
    data2 = data1.loc[data1["SPCIES_NM"] == "대파(일반)"].copy()

    # ["대파(일반)", "대파"]만 가져옴
    # 앞 뒤 공백을 제거하고 결측치는 대파로 대체함
    data2.loc[:, "CPR_USE_SPCIES_NM"] = \
        data2.loc[:, "CPR_USE_SPCIES_NM"]\
            .apply(lambda x: x.split()[0] if not pd.isna(x) else "대파")

    data3 = \
        data2.loc[
            (data2["CPR_USE_SPCIES_NM"] == "대파")
            | (data2["CPR_USE_SPCIES_NM"] == "대파(일반)")
        ].copy()

    # 수입이 포함된 값 삭제
    drop_index = data3.loc[data3["SANJI_NM"].str.contains("수입")].index
    data3.drop(drop_index, inplace=True)
    print(f"""산지에 수입이 포함된 값 {format(len(drop_index), ',')}개 삭제""")
    pprc_dict["산지에 수입이 포함된 값 삭제(A)"] = format(len(drop_index), ',')

    # 필요없는 feature 삭제
    data3.drop(
        columns=["CPR_USE_SANJI_NM", "SPCIES_NM", "CPR_USE_SPCIES_NM"], 
        inplace=True
    )

    return data3

In [54]:
def fat_finger(ds, p=10000, n=3, find_front=False):
    """
    가격이 p원 이상이고
    1의 자리에서 0이 n개 이상 지속되면 0을 하나 삭제함
    """
    def change_first_00(x):
        """
        처음 나온 0 n개에서 0을 하나 삭제함
        """
        x = str(int(x))
        find_i = x.find('0' * n)
        if find_i != -1:
            x = x[:find_i]  + x[(find_i + 1):]
        return int(x)

    temp_ds = ds.copy()
    global pprc_dict

    # 앞에서부터 0이 연속으로 나오면 
    if find_front:
        temp_index_b = temp_ds.loc[(temp_ds["PRICE"] >= p)].index
        temp_ds.loc[temp_index_b, "PRICE"] = \
            temp_ds.loc[temp_index_b, "PRICE"].apply(change_first_00)
        after_len = len(temp_ds.loc[temp_ds["PRICE"] >= p])
        print(f"fat finger로 {format(len(temp_index_b) - after_len, ',')}개 처리함")
        pprc_dict["fat finger(F) 처리(A)"] = format(len(temp_index_b) - after_len, ',')

    else:
        try:
            temp_index = \
                temp_ds.loc[
                    (temp_ds["PRICE"] >= p)
                    & (temp_ds["PRICE"] % (10 ** n) == 0)
                ].index
            temp_ds.loc[temp_index, "PRICE"] = \
                temp_ds.loc[temp_index, "PRICE"] / 10

            # 처리 정보 출력
            if len(temp_index) != 0:
                print(f"fat finger로 {format(len(temp_index), ',')}개 처리함")
                pprc_dict["fat finger(L) 처리(A)"] = format(len(temp_index), ',')

        except:
            print(f"fat finger로 처리된 값 없음")

    return temp_ds

In [55]:
def pprc_error(ds):
    data4 = ds.copy()
    global pprc_dict

    # 단위가 kg이 아닌 것 중 단위가 g인 것에서 거래단량이 100 미만인 것을
    # kg을 잘못 입력한 것으로 판단하여 kg으로 변환함
    # 즉 거래 단량이 100g 미만이면 이는 kg이라고 판단함
    # 1개 존재함
    g_error_index = data4.loc[
                        (~(data4["STNDRD"].str.contains("kg")))
                        & (data4["STNDRD"].str.contains('g'))
                        & (data4["DELNGBUNDLE_QY"] < 100)
                    ].index
    data4.loc[g_error_index, "STNDRD"] = \
        data4.loc[g_error_index, "STNDRD"].str.replace("g", 'kg')
    print(
        f"""거래 단량을 잘못 기입한 개수: {
            format(len(g_error_index), ',')
        }개"""
    )
    pprc_dict["거래 단량을 g으로 잘못 기입한 값 kg으로 대체(A)"] = \
        format(len(g_error_index), ',')

    # 단위가 g인 것을 kg으로 변환함
    g_index = data4.loc[
                (~(data4["STNDRD"].str.contains("kg")))
                & (data4["STNDRD"].str.contains('g'))
            ].index
    data4.loc[g_index, "DELNGBUNDLE_QY"] = \
        data4.loc[g_index, "DELNGBUNDLE_QY"].apply(lambda x: x / 1000)
    data4.loc[g_index, "STNDRD"] = \
        data4.loc[g_index, "STNDRD"].str.replace("g", 'kg')
    pprc_dict["거래 단량을 g인 값 kg으로 변환(A)"] = \
        format(len(g_index), ',')

    # 취소한 것 중 가격과 거래량 모두 음수이면 가격을 양수로 바꿈
    minus_index = data4.loc[
                    (data4["PRICE"] < 0)
                    & (data4["DELNG_QY"] < 0)
                ].index
    data4.loc[minus_index, "PRICE"] = \
        data4.loc[minus_index, "PRICE"].apply(lambda x: -x)

    # 취소한 것 중 가격이 음수이고 거래량이 양수이면 가격을 양수로 바꾸고 거래량은 음수로 바꿈
    p_minus_index = data4.loc[
                        (data4["PRICE"] < 0)
                        & (data4["DELNG_QY"] > 0)
                    ].index
    data4.loc[p_minus_index, "PRICE"] = \
        data4.loc[p_minus_index, "PRICE"].apply(lambda x: -x)
    data4.loc[p_minus_index, "DELNG_QY"] = \
        data4.loc[p_minus_index, "DELNG_QY"].apply(lambda x: -x)

    print(
        f"""거래가격과 거래량이 음수인 개수: {
            format(len(minus_index) + len(p_minus_index), ',')
        }개"""
    )
    pprc_dict["거래가격과 거래량이 음수인 값 거래량만 음수로 변환(A)"] = \
        format(len(minus_index) + len(p_minus_index), ',')

    # 거래 단량이 1kg 미만인 데이터의 가격을 1kg 단위로 설정함
    temp_index = \
        data4.loc[
            (data4["DELNGBUNDLE_QY"] < 1)
        ].index
    data4.loc[temp_index, "PRICE"] = \
        (
            data4.loc[temp_index, "PRICE"] / \
                data4.loc[temp_index, "DELNGBUNDLE_QY"]
        )
    pprc_dict["거래단량이 1kg 미만인 값의 단위 가격을 KRW/1kg로 변경(A)"] = \
        format(len(temp_index), ',')

    # 가격이 10,000원 이상이고, 거래단량이 1이면 fat finger로 판단하여 0을 하나 삭제함
    # 0이 2개 이상일 때만 fat finger로 판단함
    # 258개
    data5 = \
        data4.loc[
            (data4["PRICE"] >= 10000)
            & (data4["DELNGBUNDLE_QY"] == 1)
        ].copy()

    print(f"조건 전체: {format(len(data5), ',')}개 중", end=" ")

    data5 = fat_finger(data5, n=2)
    temp_index = data5.index
    data4.loc[temp_index, "PRICE"] = data5["PRICE"]

    # 0이 1개 미만이면 삭제함
    drop_index = data5.loc[data5["PRICE"] >= 10000].index
    data4.drop(drop_index, inplace=True)
    print(f"거래단량이 1인 것 중 이상치 {format(len(drop_index), ',')}개 삭제함")
    pprc_dict["거래단량이 1인 것 중 이상치 삭제(A)"] = \
        format(len(temp_index), ',')

    # # 거래단량이 0.5이하인 것은 삭제함
    # drop_index = \
    #     data4.loc[
    #         (data4["DELNGBUNDLE_QY"] < 0.5)
    #     ].index
    # data4.drop(drop_index, inplace=True)
    # print(f"{format(len(drop_index), ',')}개 삭제함")

    # 거래단량과 거래량이 이상치 값 처리
    temp_index = \
        data4.loc[
            (data4["DELNGBUNDLE_QY"] > 100)
            & (data4["DELNG_QY"] > 100)
            & (data4["DELNG_QY"] == data4["DELNGBUNDLE_QY"])
        ].index
    data4.loc[temp_index, "DELNGBUNDLE_QY"] = 1

    temp_index = \
        data4.loc[
            (data4["DELNGBUNDLE_QY"] > 1000)
            & (data4["DELNG_QY"] > 100)
        ].index
    data4.loc[temp_index, "DELNGBUNDLE_QY"] /= 1000

    return data4

In [56]:
def base_price(ds, min_q=0.3, max_q=0.7):
    """
    같은 날 거래된 거래단량이 1인 가격 중 
    
    min_q(default=30%)와 max_q(default=70%) 및 
    min_q와 max_q의 평균 가격을 반환함
    """
    data = ds.copy()
    def min_quant(x):
        return x.quantile(min_q)
    def max_quant(x):
        return x.quantile(max_q)

    # 원본 데이터의 날짜 집합
    total_d_unique = data["DELNG_DE"].unique()

    data1 = \
        data.loc[data["DELNGBUNDLE_QY"] == 1, ["DELNG_DE", "PRICE"]].copy()

    # 거래단량이 1이 존재하는 날짜의 집합
    q1_d_unique = data1["DELNG_DE"].unique()
    # 거래단량이 1이 존재하지 않는 날짜의 집합
    null_dates = np.setdiff1d(total_d_unique, q1_d_unique)

    data2 = \
        data1.groupby(
            ["DELNG_DE"], 
        ).agg({"PRICE": [
                ("MIN_Q", min_quant), 
                ("MAX_Q", max_quant),
                ("MAX", "max"),
                ("MIN", "min"),
                ]
                }
            ).copy()
    # DELNG_DE의 col_level 설정
    data2.reset_index(col_level=1, inplace=True)
    # col_level 0 삭제
    data3 = data2.droplevel(level=0, axis=1)
    data3["BASE_P"] = (data3["MIN_Q"] + data3["MAX_Q"]) / 2
    data4 = \
        data3.reindex(
            columns=["DELNG_DE", "MIN", "MIN_Q", "BASE_P", "MAX_Q", "MAX"]
        )

    # 거래단량이 1인 값이 없는 경우 전일과 후일의 평균으로 대체함.
    # 후일이 없는 경우 전일의 값으로 사용함
    data5 = data4.copy()
    for date in null_dates:
        i = 1
        while True:
            if len(data5[data5["DELNG_DE"] == date - i]) != 0:
                before_date = date - i
                break
            else:
                i += 1
        i = 1
        while True:
            if len(data5[data5["DELNG_DE"] == date + i]) != 0:
                after_date = date + i
                break
            else:
                i += 1
                if i == 10:
                    after_date = before_date
                    break
        # 전일과 후일 데이터를 가져와 평균치를 반환함
        before_d = data5[data5["DELNG_DE"] == before_date]
        after_d = data5[data5["DELNG_DE"] == after_date]
        temp = pd.concat([after_d, before_d])
        temp["DELNG_DE"] = date
        temp = temp.groupby(["DELNG_DE"], as_index=False).mean()
        data5 = pd.concat([data5, temp]).copy()
        # print(f"{date}의 기준 가격을 {before_date}와 {after_date}의 평균으로 대체함")

    # 날짜 순으로 정렬한 후 인덱스를 초기화함
    data5.sort_values("DELNG_DE", inplace=True)
    data5.reset_index(drop=True, inplace=True)

    return data5

In [57]:
def change_unit_p(ds, unit=10, min_q=0.3, max_q=0.7):
    """
    거래단량이 unit(default=10)개 미만일 때 
    단위 가격이 기준 가격보다 높을 때 
    거래단량이 2.5 이상인 경우: 단위 가격이 기준 가격의 2배 이상이면 단위가격으로 변경한다
    거래단량이 1 초과 2.5 미만인 경우: 단위 가격이 기준 가격의 1.3배 이상이면 단위가격으로 변경한다
    """
    """
    unit: 정의된 거래단량 이하일 때만 변경함
    min_q: minimum quantile
    max_q: maximum quantile
    """
    data = ds.copy()
    global pprc_dict
    
    price_df = base_price(data, min_q=min_q, max_q=max_q)
    
    # 기준가격을 데이터프레임에 합침
    data = pd.merge(data, price_df, on="DELNG_DE", how="outer")
    # 단위가격 feature 생성
    data["UNIT_P"] = data["PRICE"] / data["DELNGBUNDLE_QY"]
    # 결측치 확인
    temp_len = len(data.loc[data["BASE_P"].isna(), "DELNG_DE"].unique())
    print(f"1개 단위로 거래된 적이 없는 날의 개수: {temp_len}일")
    pprc_dict["1개 단위로 거래된 적이 없는 날의 개수(B)"] = format(temp_len, ',')
    temp_len = len(data.loc[data["BASE_P"].isna()])
    print(f"1개 단위로 거래된 적이 없는 날의 데이터의 개수: {temp_len}개")
    pprc_dict["1개 단위로 거래된 적이 없는 날의 데이터의 개수(B)"] = format(temp_len, ',')

    # 거래단량이 2.5 미만인 경우: 단위 가격이 기준 가격의 1.3배 이상이면 단위가격으로 변경한다
    # 단 변경될 단위가격은 min_q(default=30) 이상이고 max 이하여야 한다.
    temp_index = \
        data.loc[
        (data["DELNGBUNDLE_QY"] > 1)
        & (data["DELNGBUNDLE_QY"] < 2.5)
        & (data["BASE_P"].notnull())
        & (data["PRICE"] > \
            (data["BASE_P"] * data["DELNGBUNDLE_QY"] * 1.3)
          )
        & (data["UNIT_P"] >= data["MIN_Q"])
        & (data["UNIT_P"] <= data["MAX"])
        ].index
    data.loc[temp_index, "PRICE"] = \
        round(data.loc[temp_index, "PRICE"] / \
                data.loc[temp_index, "DELNGBUNDLE_QY"],
              0
        )
    print(f"거래단량이 2.5 미만인 경우 {format(len(temp_index), ',')}개 변경됨")
    pprc_dict["거래단량이 2.5 미만인 값(A)"] = \
        format(len(temp_index), ',')

    # 거래단량이 2.5 이상인 경우: 단위 가격이 기준 가격의 2배 이상이면 단위가격으로 변경한다
    # 단 변경될 단위가격은 min_q(default=30) 이상이고 max 이하여야 한다.
    temp_index = \
        data.loc[
            (data["DELNGBUNDLE_QY"] >= 2.5)
            & (data["DELNGBUNDLE_QY"] < unit)
            & (data["BASE_P"].notnull())
            & (data["PRICE"] > (data["BASE_P"] * 2))
            & (data["UNIT_P"] >= data["MIN_Q"])
            & (data["UNIT_P"] <= data["MAX"])
        ].index
    data.loc[temp_index, "PRICE"] = \
        round(data.loc[temp_index, "PRICE"] / \
              data.loc[temp_index, "DELNGBUNDLE_QY"],
              0
        )
    print(f"거래단량이 2.5 이상인 경우 {format(len(temp_index), ',')}개 변경됨")
    pprc_dict["거래단량이 2.5 이상인 값(A)"] = format(len(temp_index), ',')
    
    # 기준 가격 삭제
    data.drop(
        columns=["BASE_P", "MIN_Q", "MAX_Q", "MAX", "MIN", "UNIT_P"], 
        inplace=True,
    )

    return data

In [58]:
def check_p_range(ds, min_q=0.2, max_q=0.8, price=None):
    """
    price: "unit" => 단위가격 사용

    같은 날 거래된 가격 중 price가

    min_q(default=20%)와 max_q(default=80%) 사이에 있으면
    
    TRUE를 반환한다
    """
    data = ds.copy()

    # 같은 날 거래된 가격 중 MIN_Q와 MAX_Q를 구한다.
    price_df = base_price(data, min_q=min_q, max_q=max_q)
    price_df.drop(columns=["MIN", "BASE_P", "MAX"], inplace=True)
    data = pd.merge(data, price_df, on="DELNG_DE", how="outer")
    # 가격이 MIN_Q와 MAX_Q 사이에 있으면 TRUE를 반환한다.
    if price == "unit":
        data["UNIT_P"] = data["PRICE"] / data["DELNGBUNDLE_QY"]
        p = "UNIT_P"
    else:
        p = "PRICE"
    
    data["Q_RANGE"] = (
        (data["MIN_Q"] <= data[p])
        & (data[p] <= data["MAX_Q"])
    )
    if price == "unit":
        data.drop(columns=["MIN_Q", "MAX_Q", "UNIT_P"], inplace=True)
    else:
        data.drop(columns=["MIN_Q", "MAX_Q"], inplace=True)
    
    return data

In [59]:
def pprc_price(ds, price=9000, weight=1.3):
    """
    가격 이상치를 처리함

    price: 를 초과하는 가격일 때 이상치로 판단함

    weight: 거래단량이 1인 것의 MAX 값에 곱할 가중치
    """
    global pprc_dict
    data = ds.copy()

    # 단위가격이 quantile 범위 내에 드는지 확인하는 feature 'Q_RANGE'생성
    data = check_p_range(data, min_q=0.2, max_q=0.8, price="unit")

    # 가격이 9,000원 이상이고 Q_RANGE가 TRUE이면 단위가격으로 변경함
    temp_index = \
        data.loc[
            (data["Q_RANGE"])
            & (data["PRICE"] > price)
        ].index
    data.loc[temp_index, "PRICE"] = \
        data.loc[temp_index, "PRICE"] / data.loc[temp_index, "DELNGBUNDLE_QY"]
    print(f"단위가격으로 {format(len(temp_index), ',')}개 처리함")
    pprc_dict["단위가격으로 처리(A)"] = format(len(temp_index), ',')

    # 같은 날 거래된 가격 중 거래단량이 1인 것의 MAX를 구한다.
    price_df = base_price(data)
    price_df.drop(columns=["MIN_Q", "BASE_P", "MAX_Q"], inplace=True)
    data = pd.merge(data, price_df, on="DELNG_DE", how="outer")

    # 가격이 9,000원 이상이고 Q_RANGE가 FALSE이며,
    # 단위가격이 MAX * weight(default=1.3)을 초과할 때 
    # 0이 2개 연속이면 fat_finger로 처리함
    data["UNIT_P"] = data["PRICE"] / data["DELNGBUNDLE_QY"]
    temp_ds = \
        data.loc[
            ~(data["Q_RANGE"])
            & (data["PRICE"] > price)
            & (data["UNIT_P"] > data["MAX"] * weight)
        ]
    temp_index = temp_ds.index
    temp_ds = fat_finger(temp_ds, p=price, n=2)
    data.loc[temp_index] = temp_ds

    # 가격이 9,000원 이상이고 Q_RANGE가 FALSE일 때 
    # 단위가격이 MIN보다 크거나 같고 MAX * weight + 0.5(default=1.8) 이내이면
    # 단위가격으로 변경함
    temp_index = \
        data.loc[
            ~(data["Q_RANGE"])
            & (data["PRICE"] > price)
            & (data["UNIT_P"] >= data["MIN"])
            & (data["UNIT_P"] <= data["MAX"] * (weight + 0.5))
        ].index
    data.loc[temp_index, "PRICE"] = data.loc[temp_index, "UNIT_P"]
    print(f"단위가격으로 {format(len(temp_index), ',')}개 처리함")
    pprc_dict["단위가격으로 처리(A)"] = format(len(temp_index), ',')

    data = check_p_range(data, min_q=0.2, max_q=0.8, price="unit")
    # 가격이 9,000원 이상이고 Q_RANGE가 FALSE이며, 
    # 앞에서부터 0이 2개 연속이면 fat_finger로 처리함
    data["UNIT_P"] = data["PRICE"] / data["DELNGBUNDLE_QY"]
    temp_ds = \
        data.loc[
            ~(data["Q_RANGE"])
            & (data["PRICE"] > price)
        ]
    temp_index = temp_ds.index
    temp_ds = fat_finger(temp_ds, p=price, n=2, find_front=True)
    data.loc[temp_index] = temp_ds
    
    # 처리되지 않은 값은 삭제함
    drop_index = \
        data.loc[
            ~(data["Q_RANGE"])
            & (data["PRICE"] > price)
        ].index
    data.drop(drop_index, inplace=True)
    print(f"{format(len(drop_index), ',')}개 삭제함")
    pprc_dict["이상치 처리가 안 되는 값 삭제(A)"] = format(len(drop_index), ',')

    data.drop(columns=["Q_RANGE", "MIN", "MAX", "UNIT_P"], inplace=True)

    return data

In [60]:
# 창원내서도매시장 이상치로 인해 drop
drop_index = select_data[
    select_data["PBLMNG_WHSAL_MRKT_NM"] == "창원내서도매시장"
].index
data0 = select_data.drop(drop_index).copy()

data1 = pprc_default(data0)
data2 = pprc_error(data1)
data3 = change_unit_p(data2, unit=float("inf"))
data4 = change_unit_p(data3, unit=float("inf"), min_q=0.01, max_q=0.01)
data5 = pprc_price(data4, price=9000, weight=1.3)

산지가 없는 값 비율: 2.58%
산지가 없는 값 개수: 64,530개
산지가 없는 값 9,068개 대체
결측치 처리 후 산지가 없는 값 개수: 55,462개 삭제
가격, 거래단량, 거래량이 하나라도 0인 값 672개 삭제
대파(일반)의 비율: 95.39%
산지에 수입이 포함된 값 3,655개 삭제
거래 단량을 잘못 기입한 개수: 1개
거래가격과 거래량이 음수인 개수: 1,373개
조건 전체: 256개 중 fat finger로 202개 처리함
거래단량이 1인 것 중 이상치 54개 삭제함
1개 단위로 거래된 적이 없는 날의 개수: 0일
1개 단위로 거래된 적이 없는 날의 데이터의 개수: 0개
거래단량이 2.5 미만인 경우 91,257개 변경됨
거래단량이 2.5 이상인 경우 351,985개 변경됨
1개 단위로 거래된 적이 없는 날의 개수: 0일
1개 단위로 거래된 적이 없는 날의 데이터의 개수: 0개
거래단량이 2.5 미만인 경우 553,162개 변경됨
거래단량이 2.5 이상인 경우 249,267개 변경됨
단위가격으로 83개 처리함
fat finger로 334개 처리함
단위가격으로 4,470개 처리함
fat finger로 295개 처리함
300개 삭제함


In [61]:
# 전처리 된 항목 파일로 저장
pprc_df = pd.DataFrame([pprc_dict.keys(), pprc_dict.values()]).T
pprc_df.columns = ["전처리 항목", "전처리 된 개수"]
pprc_df.to_csv("전처리된_항목_ver3.0.csv")

# 전처리 된 후 값만 출력
for key in pprc_dict:
    if "(B)" not in key:
        key_temp = key.replace("(A)", '')
        print(f"{key_temp}: {pprc_dict[key]}")

# 전처리 되기 전 값만 출력
for key in pprc_dict:
    if "(A)" not in key:
        key_temp = key.replace("(B)", '')
        print(f"{key_temp}: {pprc_dict[key]}")

산지가 없는 값 대체: 9,068
산지가 없는 값 삭제: 55,462
가격, 거래단량, 거래량이 하나라도 0인 값 삭제: 672
대파(일반) 아닌 값 삭제: 113,039
산지에 수입이 포함된 값 삭제: 3,655
거래 단량을 g으로 잘못 기입한 값 kg으로 대체: 1
거래 단량을 g인 값 kg으로 변환: 19,088
거래가격과 거래량이 음수인 값 거래량만 음수로 변환: 1,373
거래단량이 1kg 미만인 값의 단위 가격을 KRW/1kg로 변경: 60,171
fat finger(L) 처리: 334
거래단량이 1인 것 중 이상치 삭제: 256
거래단량이 2.5 미만인 값: 553,162
거래단량이 2.5 이상인 값: 249,267
단위가격으로 처리: 4,470
fat finger(F) 처리: 295
이상치 처리가 안 되는 값 삭제: 300
Raw data 개수: 2,505,639
산지가 없는 값: 64,530
1개 단위로 거래된 적이 없는 날의 개수: 0
1개 단위로 거래된 적이 없는 날의 데이터의 개수: 0


In [62]:
data5["VOLUME"] = data5["DELNG_QY"] * data5["DELNGBUNDLE_QY"]
data5.to_csv(EDA_FILE_NAME)

In [63]:
def loca_change(ds):
    """
    시군구를 정의된 양식에 따라 변경함
    """

    data = ds.copy()

    global error_list
    error_list = set()
    sido_dict = {
        "경기도": "경기",
        "강원도": "강원",
        "경상북도": "경북",
        "경상남도": "경남",
        "충청북도": "충북",
        "충청남도": "충남",
        "전라북도": "전북",
        "전라남도": "전남",
        "서울특별시": "서울",
        "세종특별시": "세종",
        "부산광역시": "부산",
        "울산광역시": "울산",
        "대구광역시": "대구",
        "대전광역시": "대전",
        "제주특별자치도": "제주",
        "제주도": "제주",
        "광주광역시": "광주",
        "인천광역시": "인천",
    }

    def check_si_gun_gu(text):
        """
        시/군/구가 포함되어 있는지 확인한다.
        """
        if (text[-1] == "시") | (text[-1] == "군") | (text[-1] == "구"):
            return True
        else:
            return False

    def check_double(text):
        """
        광역자치단체 명이 중복되어 있는지 확인한다.
        """
        temp_list = sido_dict.values()
        if text in temp_list:
            return True
        else:
            return False

    def loca_name_change(data):
        """
        광역자치단체 명과 시군구 명을 통일한다.
        """

        temp = list(filter(None, data.split(" ")))
        
        # 광역시도명 짧게 변경
        if temp[0] in sido_dict.keys():
            temp[0] = sido_dict[temp[0]]
        
        # 시군구 짧게 변경
        if len(temp) >= 2:
            if check_double(temp[1]):
                temp[1] = temp[2]

            if check_si_gun_gu(temp[1]): 
                res = temp[0] + " " + temp[1][:-1] # 시군구 삭제하고 결합
            else:
                res = temp[0] + " " + temp[1]
        else:
            res = temp[0]

        return res

    def merge_loca(data):
        """
        사전에 정의한 지리적으로 근접한 지역을 합침.
        """
        def exception_process(text):
            """
            예외적인 지역 이름을 처리함.
            """
            # 리스트로 들어오면 문자열로 변경함
            text = ' '.join(text)
            exception_dict = {"완주": "전라남도(진도+)",
                            }

            # 시군구가 포함되어 있으면 시군구로 변경함
            for area in area_dict.keys():
                if area in text:
                    text = area_dict[area]
                    # 종료
                    return text

            # 예외 처리 항목에 포함되어 있으면 해당 값으로 변경함
            for area in exception_dict.keys():
                if area in text:
                    text = exception_dict[area]
                    return text
            
            # 광역지자체가 포함되어 있으면 광역지자체로 변경함
            for area in metro_area_dict.keys():
                if area in text:
                    text = metro_area_dict[area]
                    return text
            
            # # 처리되지 않았으면 메세지 출력함 
            # print('='*79)
            # print(f"{text} 처리 필요함!!!".center(60, ' '))
            # print('='*79)

            # 처리되지 않은 지역 리스트 추가
            global error_list
            error_list.update([text])
            
            return text

        area_dict = {
            # "신안": "전라남도(진도+)",
            # "진도": "전라남도(진도+)",
            # "해남": "전라남도(진도+)",
            # "안양": "경기도(구리+)",
            # "구리": "경기도(구리+)",
            # "남양주": "경기도(구리+)",
            # "하남": "경기도(구리+)",
            # "성남": "경기도(구리+)",
            # "광주": "경기도(구리+)",
            # "수원": "경기도(구리+)",
            # "안성": "경기도(구리+)",
            # "오산": "경기도(구리+)",
            # "평택": "경기도(구리+)",
            # "이천": "경기도(구리+)",
            # "여주": "경기도(구리+)",
            # "양평": "경기도(구리+)",
            # "용인": "경기도(구리+)",
            # "화성": "경기도(구리+)",
            # "고양": "경기도(구리+)",
            # "파주": "경기도(구리+)",
            # "양주": "경기도(구리+)",
            # "동두천": "경기도(구리+)",
            # "연천": "경기도(구리+)",
            # "포천": "경기도(구리+)",
            # "가평": "경기도(구리+)",
            # "의정부": "경기도(구리+)",
            # "김포": "경기도(구리+)",
            # "부천": "경기도(구리+)",
            # "안산": "경기도(구리+)",
            # "군포": "경기도(구리+)",
            # "광명": "경기도(구리+)",
            # "시흥": "경기도(구리+)",
            # "평창": "강원도(평창+)",
        }
        metro_area_dict = {
            "경남": "경상남도(부산+)",
            "부산": "경상남도(부산+)",
            "울산": "경상남도(부산+)",
            "전남": "전라남도(진도+)",
            "광주": "전라남도(진도+)",
            "경기": "경기도(구리+)",
            "강원": "강원도(평창+)",
            # "전북": "전라북도",
            # "충북": "충청북도 청주+",
            # "충남": "충청남도 아산+",
            # "경북": "경상북도 구미+(+대구)",
            # "대전": "충청북도 청주+",
            # "세종": "충청북도 청주+",
            # "대구": "경상북도 구미+(+대구)",
            # "서울": "경기도(구리+)",
            # "인천": "경기도(구리+)",
        }

        temp = list(filter(None, data.split(" ")))
        
        if len(temp) >= 2:
            if temp[1] in area_dict.keys():
                res = area_dict[temp[1]]
            elif temp[0] in metro_area_dict.keys():
                res = metro_area_dict[temp[0]]
            else:
                res = exception_process(temp)
        else:
            if temp[0] in metro_area_dict.keys():
                res = metro_area_dict[temp[0]]
            else:
                res = exception_process(temp)

        return res

    data.loc[:, "SANJI_NM"] = data.loc[:, "SANJI_NM"].apply(loca_name_change)

    # 생산량이 미미한 지역 제거함
    drop_loca_list = \
        ["제주", "전북", "충북", "충남", "경북", "대전", "세종", "대구", "서울", "인천"]
    for d_loca in drop_loca_list:
        drop_index = data[data["SANJI_NM"].str.contains(d_loca)].index
        data.drop(drop_index, inplace=True)

    # 산지 명을 정의한대로 설정
    data.loc[:, "SANJI_NM"] = data.loc[:, "SANJI_NM"].apply(merge_loca)

    # 정의되지 않은 지역 리스트 출력
    print(f"처리가 필요한 지역 리스트: {error_list}")
    for er_loca in error_list:
        drop_index = data[data["SANJI_NM"].str.contains(er_loca)].index
        data.drop(drop_index, inplace=True)

    # 정의된 지역의 개수 = 13개
    # data["SANJI_NM"].unique()
    # data["SANJI_NM"].value_counts()
    print(f"""데이터 내 지역의 개수: {data["SANJI_NM"].nunique()}개""")

    return data


In [64]:
data6 = loca_change(data5)
data6.to_csv(POP_LOCA_FILE_NAME)

처리가 필요한 지역 리스트: {'해외', '뉴질랜드', '스위스', '중국'}
데이터 내 지역의 개수: 4개


In [65]:
# ##############################################################################
# # 시군구 기준 거래량 상위 50개 지역
# ##############################################################################
# group_temp = data_t.loc[:, ["SANJI_NM", "거래량*kg"]].groupby("SANJI_NM").sum()
# total_q = int(data_t[["거래량*kg"]].sum().values)
# group_temp["ratio"] = \
#     group_temp["거래량*kg"].apply(lambda x: format(x / total_q * 100, ".2f"))

# print('=' * 79)
# print("시군구 기준 거래량 상위 50개 지역")
# print('=' * 79)
# print(group_temp.sort_values(by="거래량*kg", ascending=False).head(50))
# print('=' * 79, end="\n\n\n")
# ##############################################################################
# ##############################################################################


# ##############################################################################
# # 지역 기준 거래량 상위 지역
# district = "경기"
# ##############################################################################
# group_temp = group_temp.reset_index()

# # print('=' * 79)
# # print(f"{district} 기준 거래량 상위 50개 지역")
# # print('=' * 79)
# # print(group_temp.loc[group_temp.SANJI_NM.str.contains(district)]\
# #     .sort_values(by="거래량*kg", ascending=False))
# # print('=' * 79, end="\n\n\n")
# ##############################################################################
# ##############################################################################

# ##############################################################################
# # 광역지자체 기준 거래량 상위 지역
# ##############################################################################
# def pop_sido(data):
#     temp = list(filter(None, data.split(" ")))
#     res = temp[0]
#     return res    

# group_temp["SANJI_NM"] = group_temp["SANJI_NM"].apply(pop_sido)
# group_temp = group_temp.groupby("SANJI_NM").sum()
# group_temp["ratio"] = \
#     group_temp["거래량*kg"].apply(lambda x: format(x / total_q * 100, ".2f"))

# print('=' * 79)
# print("광역 지자체 기준 거래량 상위 지역")
# print('=' * 79)
# print(group_temp.sort_values(by="거래량*kg", ascending=False))
# print('=' * 79, end="\n\n\n")
# ##############################################################################
# ##############################################################################